In [ ]:
import os
import openai
from dotenv import load_dotenv

os.chdir('/workspace')

# Load environment variables
if load_dotenv():
    print("Found OpenAPI Base Endpoint: " + os.getenv("OPENAI_API_BASE"))
else: 
    print("No file .env found")

openai_api_type = os.getenv("OPENAI_API_TYPE")
openai_api_key = os.getenv("OPENAI_API_KEY")
openai_api_base = os.getenv("OPENAI_API_BASE")
openai_api_version = os.getenv("OPENAI_API_VERSION")
deployment_name = os.getenv("AZURE_OPENAI_COMPLETION_DEPLOYMENT_NAME")
embedding_name = os.getenv("AZURE_OPENAI_EMBEDDING_DEPLOYMENT_NAME")

def get_completion(prompt, model=deployment_name, temperature=0.0):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        engine=model,
        messages=messages,
        temperature=temperature, # this is the degree of randomness of the model's output
    )
    return response.choices[0].message["content"]


## Prompting Principles
- **Principle 1: Write clear and specific instructions**
- **Principle 2: Give the model time to “think”**



In [ ]:
text = """
At four hours by plane, you arrive at the Spanish Gran Canaria, 
one of the most versatile Canary Islands. Gran Canaria has many 
attractions such as Roque Nublo and Dunas de Maspalomas. 
In addition, there are several picturesque villages. So there is 
always something to do! Curious about the activities you can prepare 
for? Read all the highlights of Gran Canaria in this blog 
and let your holiday anticipation begin!

Explore attractions with your rental car
Gran Canaria is perfect for exploring by car. The GC-60 is the 
access road to Roque Nublo, a rock formation that is 80 meters high. 
It is the symbol of the island and is also called the "Grand Canyon" 
of Gran Canaria. It is one of the most beautiful attractions because 
you can hike and walk among the rugged rocks. So bring good walking 
shoes. It feels like you're walking on the moon here, so special!

Then drive to the mountain village of Fataga. An oasis of tranquility 
with typical indigenous Canarian architecture: white houses with 
orange-red roofs and narrow streets. The highlight is the view. 
On one side, you can see the "Grand Canyon" and on the other side, 
you have a view of the ocean. The capital Las Palmas is also well 
worth a visit. The city is full of attractions such as a large harbor, 
cozy markets, and many shopping opportunities. But the city also offers 
the opportunity to enjoy the beach.

Gran Canaria: sun, sea, and beach
What to do in Gran Canaria? Relax with the sand between your toes! 
Every Sunweb travel guide mentions Playa Amadores as one of the 
most beautiful beaches in Gran Canaria. Of course, you want to 
see that with your own eyes. Admire the beautiful bay with clear 
blue water, fine sand, and swaying palm trees. Just the palm trees 
alone give you that holiday feeling! There are also many restaurants 
and souvenir shops in Playa Amadores. Near this beach is Puerto Rico, 
a sunny seaside resort with a beautiful beach and cozy terraces and shops. 
From here, you can easily reach the charming Puerto de Mogán by car 
or small ferry. This fishing village has a lively marina with colorful 
boats and a lovely child-friendly beach. Nice to relax and cool off 
in the clear water!


"""
prompt = f"""
Summarize the text delimited by triple backticks
into a single sentence.
```{text}```
"""
response = get_completion(prompt)
print(response)

#### Tactic 1: Ask for structured output

In [ ]:
prompt = """
Generate a list of three made-up holiday destinations in Greece 
with location, average summer temperatures and three main attractions.
Provide them in the following format:

Destination name:
Destination location:
Average summer temperature:
Three main attractions:
    - 
    -
    -
"""
response = get_completion(prompt, temperature=0.9)
print(response)

**Additional tasks:**

* Try adding additional fields to be included or provide additional context to the prompt to generate more contextual output.

* Pass temperature parameter to control the randomness of the output. The higher the temperature, the more random the output. (e.g. `get_completion(prompt, temperature=0.9)`)

#### Tactic 2: Ask the model to check whether conditions are satisfied

In [ ]:
text_1 = """
Making a cup of tea is easy! First, you need to get some
water boiling. While that's happening, 
grab a cup and put a tea bag in it. Once the water is 
hot enough, just pour it over the tea bag. 
Let it sit for a bit so the tea can steep. After a 
few minutes, take out the tea bag. If you 
like, you can add some sugar or milk to taste.
And that's it! You've got yourself a delicious
cup of tea to enjoy.
"""

prompt = f"""
You will be provided with text delimited by triple quotes.
If it contains a sequence of instructions,
re-write those instructions in the following format:

Step 1 - ...
Step 2 - …
…
Step N - …

If the text does not contain a sequence of instructions,
then simply write \"No steps provided.\"

\"\"\"{text_1}\"\"\"
"""
response = get_completion(prompt)
print("Completion for Text 1:")
print(response)

In [ ]:
text_2 = """
Do you want to travel extra comfortably? Or do you want
to sit next to your children, for example? Or are you tall
and do you like to book extra legroom? Choose and reserve
your favorite seat yourself before departure. The seat
reservation is provided by the airline you are going on
vacation with. Sunweb has no influence on the seat reservation.
It is not possible to reserve a seat with all airlines.
Below you will find all the airlines we work with and the
information you need to reserve a seat.
"""
prompt = f"""
You will be provided with text delimited by triple quotes.
If it contains a sequence of instructions, \
re-write those instructions in the following format:

Step 1 - ...
Step 2 - …
…
Step N - …

If the text does not contain a sequence of instructions, \
then simply write \"No steps provided.\"

\"\"\"{text_2}\"\"\"
"""
response = get_completion(prompt)
print("Completion for Text 2:")
print(response)

### Principle 2: Give the model time to “think” 

#### Tactic 1: Specify the steps required to complete a task

In [ ]:
text = """
This special hotel offers you chic rooms & suites that are fully equipped.
Each room has its own charm and features+ beautiful oak floors, very comfortable beds\
and luxurious bathrooms. One of the most beautiful parts of the hotel is the spa.\
This fantastic spa is located on the ground floor and from the spa you have an amazing \
view of the mountains and the slopes. Here you will find a sauna, herbal sauna and a Turkish steam bath.
"""
# example 1
prompt_1 = f"""
Perform the following actions: 
1 - Translate the summary into Dutch.
2 - List all hotel characteristics in Dutch.
3 - Translate the characteristics into Catalan.
4 - List all hotel characteristics in Catalan.
5 - Structure output in the following way
dutch_summary:
dutch_characteristics:
catalan_summary:
catallan_characteristics:


Provide only the output from task number 5

Text:
```{text}```
"""
response = get_completion(prompt_1)
print("Completion for prompt 1:")
print(response)

#### Tactic 2: Instruct the model to work out its own solution before rushing to a conclusion

In [ ]:
prompt = """
Determine if the student's solution is correct or not.

Question:
I'm building a solar power installation and I need \
 help working out the financial. 
- Land costs $100 / square foot
- I can buy solar panels for $250 / square foot
- I negotiated a contract for maintenance that will cost \
me a flat $100k per year, and an additional $10 / square \
foot
What is the total cost for the first year of operations 
as a function of the number of square feet.

Student's Solution:
Let x be the size of the installation in square feet.
Costs:
1. Land cost: 100x
2. Solar panel cost: 250x
3. Maintenance cost: 100,000 + 100x
Total cost: (100 * x)  + (250 * x) + (100,000 + (100 * x)) = 450x + 100,000
"""
response = get_completion(prompt)
print(response)

***Note that the student's solution is actually not correct.***
#### We can fix this by instructing the model to work out its own solution first.

In [ ]:
prompt = """
Your task is to determine if the student's solution \
is correct or not.
To solve the problem do the following:
- First, work out your own solution to the problem. 
- Then compare your solution to the student's solution \
and evaluate if the student's solution is correct or not. 
Don't decide if the student's solution is correct until 
you have done the problem yourself.

Use the following format:
Question:
```
question here
```
Student's solution:
```
student's solution here
```
Actual solution:
```
steps to work out the solution and your solution here
```
Is the student's solution the same as actual solution \
just calculated:
```
yes or no
```
Student grade:
```
correct or incorrect
```

Question:
```
I'm building a solar power installation and I need help \
working out the financial. 
- Land costs $100 / square foot
- I can buy solar panels for $250 / square foot
- I negotiated a contract for maintenance that will cost \
me a flat $100k per year, and an additional $10 / square \
foot
What is the total cost for the first year of operations \
as a function of the number of square feet.
``` 
Student's solution:
```
Let x be the size of the installation in square feet.
Costs:
1. Land cost: 100x
2. Solar panel cost: 250x
3. Maintenance cost: 100,000 + 100x
Total cost: (100 * x)  + (250 * x) + (100,000 + (100 * x)) = 450x + 100,000
```
Actual solution:
"""
response = get_completion(prompt)
print(response)

## Model Limitations: Hallucinations
- As you know Sunweb doesn't offer something called `regenvakantie` but still the model will generate something.

In [ ]:
prompt = """
Tell something about regenvakantie at Sunweb
"""
response = get_completion(prompt, temperature=0.9)
print(response)

# Iterative Prompt Develelopment
In this section, you'll iteratively analyze and refine your prompts to generate marketing copy from a product fact sheet.


## Generate a marketing destination description from destination fact sheet

In [ ]:
tenerife_fact_sheet = """

- Tenerife is the largest of the Canary Islands.
- The Tenerife flag is the same as Scotland’s, this is because St Andrew is the patron saint of the island.
- 43% of the entire Canary Islands population live on the island.
- The canary bird was named after the islands, not the other way around.
- Every year the island attracts over five million tourists.
- You will always find a spot to lay your towel with a trip to El Medano – Tenerife’s longest beach stretching out just over two kilometers.
- Many of Tenerife’s beaches are not natural, but man-made due to the islands volcanic nature.\
  You will find the natural ones have characteristic back sand.
- The popular holiday destination Playa de Las Americas can be reached by both of the international airports:\
  Reina Sofia in the south of the Island and Los Cristianos. \
  Los Rodeos airport north of the island is near the tourist resort of Puerto de La Cruz.
- Two of the biggest attractions on the island are Loro Parque in Puerto de la Cruz and the volcano Teide\
  the top of which is more than 3,000 meters above sea level.
- Tenerife’s Teide National Park is a UNESCO World Heritage Site and the second most visited park in the entire world.\
- Due to its colossal size, Mount Teide is known to cast the largest sea shadow in the world.
- The island’s famous Thai themed Siam Park, is the biggest water park in Europe and offers one of the highest water slides in the world.
- The ‘Wind Cave’ (La Cueva del Viento) is the largest volcanic tube in Europe measuring 18 kilometers in length.\

"""

In [ ]:
prompt = f"""
Your task is to help a marketing team 
of tour operating company create a 
destination description for the company's website.

Write a destination description based on the information 
provided in the fact sheet delimited by 
triple backticks.

Fact sheet: ```{tenerife_fact_sheet}```
"""
response = get_completion(prompt)
print(response)

## Issue 1: The text is too long 
- [Action Needed] Update the prompt to limit the number of words/sentences/characters.

In [ ]:
prompt = f"""
Your task is to help a marketing team 
of tour operating company create a 
destination description for the company's website.

Write a destination description based on the information 
provided in the fact sheet delimited by 
triple backticks.

Fact sheet: ```{tenerife_fact_sheet}```
"""
response = get_completion(prompt)
print(response)

## Issue 2. Text focuses on the wrong details
- Ask it to focus on the aspects that are relevant to the intended audience.

In [ ]:
prompt = f"""
Your task is to help a marketing team 
of tour operating company create a 
destination description for the company's website.

Write a destination description based on the information 
provided in the fact sheet delimited by 
triple backticks.

The description should be written in a way that is attractive
for audience looking for active holidays. Write a description
in a style of friend sharing recommendation with another friend.
Your answer must not be longer than 50 words so choose only top 3
recommendations that will be attractive for active holiday seekers. 

Fact sheet: ```{tenerife_fact_sheet}```
"""
response = get_completion(prompt)
print(response)

**Additional tasks:**

* Experiment with prompts that focus on target audience to be elderly people or families with children.
* Experiment with prompts that focus on the destination's natural beauty or cultural heritage.

# Inferring
In this section, you will infer sentiment and topics from product reviews and news articles.


In [ ]:
review_one = """
That girl barely had any product knowledge, preferred to look\
the other way when we meet (they stayed at the hotel).\
Even basic politeness was lacking. In recent years we have noticed more and\
more that there is much less experience and interest in other things than the\
well-being of the customer. After many and many years of Sunweb, \
it's time for us to take a look at the competition. \
Too bad, put a damper on our holiday. Because of Sunweb's standard responses here,\
I'm not going to elaborate either. If necessary, you know where to find us for more feedback.\
Last year we noticed in Gouves, Santa Susanna and now also Rhodes that the well-being\
of the customer and interest in their product no longer come first. Shame!
The only positive aspect of this holiday was quality of food in the hotel.
"""

In [ ]:
review_two = """
Our trip to Crete was booked smoothly from the 1st attempt, after which there was\
good and clear communication for further steps. Upon arrival, Sunweb employees were \
already at the airport for a transfer. The hotel SOLIMAR DIAS was also a good and friendly\
reception with a good location, warm and homely atmosphere. REALLY RECOMMENDED. I will definitely go back
"""

In [ ]:
prompt = f"""
What is the sentiment of the following product review, 
which is delimited with triple backticks?

Review text: '''{review_two}'''
"""
response = get_completion(prompt)
print(f'Review Text: {review_two}')
print(f'Response: {response}')

In [ ]:
prompt = f"""
Identify a list of emotions that the writer of the \
following review is expressing. Include no more than \
five items in the list. Format your answer as a list of \
lower-case words separated by commas.

Review text: '''{review_one}'''
"""
response = get_completion(prompt)
print(f'Review Text: {review_one}')
print(f'Response: {response}')

In [ ]:
prompt = f"""
Is the writer of the following review expressing anger?\
The review is delimited with triple backticks. \
Give your answer as either yes or no.

Review text: '''{review_two}'''
"""
response = get_completion(prompt)
print(f'Review Text: {review_two}')
print(f'Response: {response}')

In [ ]:
prompt = f"""
Determine list of topics that were mentioned in the review.
Per each topic, classify if the sentiment associated with it\
is positive, negative or neutral

Format your response as a list of items separated by commas.

Text sample: '''{review_one}'''
"""
response = get_completion(prompt)
print(response)

# Bring your own data and LangChain

In this section, we'll explore how we can bring our own data into the models used by Azure OpenAI and introduce [LangChain](https://python.langchain.com/docs/get_started/introduction), a framework for developing applications powered by language models.

Langchain supports Python and Javascript / Typescript. For this lab, we will use Python.

We'll start by importing the AzureOpenAI specific components from the langchain package, including models and schemas for interacting with the API.

In [ ]:
from langchain.llms import AzureOpenAI
from langchain.chat_models import AzureChatOpenAI
from langchain.schema import HumanMessage

Next, we'll configure Langchain by providing the API key and endpoint details, along with the API version information.

As Langchain can work with multiple AI services, we need to specify that we want to work with Azure via the openai_api_type parameter.

In [ ]:
# Create an instance of Azure OpenAI
llm = AzureChatOpenAI(
    openai_api_type = openai_api_type,
    openai_api_version = openai_api_version,
    openai_api_base = openai_api_base,
    openai_api_key = openai_api_key,
    deployment_name = deployment_name,
    temperature = 0
)

Let's begin by asking the AI a simple question.

In [ ]:
# Define the prompt we want the AI to respond to - the message the Human user is asking
msg = HumanMessage(content="Tell me about the latest Ant-Man movie. When was it released? What is it about?")

# Call the AI
r = llm(messages=[msg])

# Print the response
print(r.content)

What do you notice about the response?

The AI thinks the latest "Ant-Man" movie was "Ant-Man and the Wasp" and it was released in July 2018. This is wrong, as there has been a more recent "Ant-Man" movie.

OpenAI models are trained on a large set of data, but that happened at a specific point in time depending on the model. So, many of the models have no information about events that took place in recent months or years.

To help the AI out, we can provide additional information. This is the same process you would follow if you want the AI to work with your own company data. The AI won't know about information that isn't publically available, so if you want the AI to work with that information, then you'll need to get that information into the model.

The thing is, you can't actually do that. The models are pre-trained, so the only way to get more information in is to retrain the model, which is an expensive and time consuming process.

However, there are ways to get the AI models to work with new data. The most popular of these methods is to use embeddings, which we'll explore in the next sections.

### Bring Your Own Data

Langchain provides a number of useful tools, which include tools to simplify the process of working with external documents. Below, we'll use the `DirectoryLoader` which can read multiple files from a directory and the `UnstructuredMarkdownLoader` which can process files in Markdown format. We'll use these to process a bunch of markdown formatted files that contain details of movies that were released in the year 2023.

In [ ]:
from langchain.document_loaders import DirectoryLoader, UnstructuredMarkdownLoader

data_dir = "data/movies"

documents = DirectoryLoader(path=data_dir, glob="*.md", show_progress=True, loader_cls=UnstructuredMarkdownLoader).load()

We now have a `documents` object which contains all of the information from our markdown documents about movies.

We can use the `question_answering` chain to provide the AI with access to our documents and then ask the same question about Ant-Man movies again.

In [ ]:
# Question answering chain
from langchain.chains.question_answering import load_qa_chain

# Prepare the chain and the query
chain = load_qa_chain(llm)
query = "Tell me about the latest Ant Man movie. When was it released? What is it about?"

chain.run(input_documents=documents, question=query)

Great! The model now knows about the latest Ant-Man movie.

However, there's something lurking! Let's take a look at what happened behind the scenes.

We'll do two things here. First we'll add the `verbose=True` parameter to the chain, and we'll wrap the chain execution in a callback, which will allow us to capture the number of tokens consumed.

In [ ]:
# Support for callbacks
from langchain.callbacks import get_openai_callback

# Prepare the chain and the query
chain = load_qa_chain(llm, verbose=True)
query = "Tell me about the latest Ant Man movie. When was it released? What is it about?"

# Run the chain, using the callback to capture the number of tokens used
with get_openai_callback() as callback:
    chain.run(input_documents=documents, question=query)
    total_tokens = callback.total_tokens

print(f"Total tokens used: {total_tokens}")

In the output from the last code section, you should see a lot of information. At the end, you should see a count of the number of tokens used. You might be surprised to see that the query uses something like 2,500 tokens! That's a lot of tokens.

With the verbose option enabled, the rest of the output shows the prompt that was constructed for the query. If you scroll back through the output, you'll see that the prompt included all of the information from our documents, so this is why the query used so many tokens.

As we've discussed previously, AI models have a maximum number of tokens you can use and a charging model based on the number of tokens consumed. In this example, the documents are relatively small in size and there's only 20 of them, so clearly this is not going to scale when we want to work with larger documents and more of them.

### Embeddings

The solution to working with large amounts of external information is to use embeddings. OpenAI provide embedding models which allow human readable information to be analysed for meaning and intent. The output from an embedding model is data in a numeric format, known as vectors. These allow computers to group pieces of similar information together. The vectors are then kept in a vector store. When you want to ask a question, an embedding model is again used to convert the query text into vectors and the vector data that represents your query can then be searched in the vector store. Any similar vectors that are found in the database are likely to be a good response to your query.

To prevent overloading a prompt with a large number of tokens, instead of sending all of our documents to the AI, we can perform a vector search first to narrow down to a set of interesting results, and then use that smaller subset of information as part of a prompt.

Let's walk through the process of using embeddings to give the AI some details about our movies. We'll start by initiating an instance of an embeddings model. You'll notice this is similar to when we initialise one of our model deployments to run a query, but in this case we specify and embedding model. Typically the embedding model used is `text-embedding-ada-002`.

In [ ]:
from langchain.embeddings import OpenAIEmbeddings

embeddings_model = OpenAIEmbeddings(
    openai_api_type = openai_api_type,
    openai_api_version = openai_api_version,
    openai_api_base = openai_api_base,
    openai_api_key = openai_api_key,
    deployment = embedding_name
)

Now that we've initialised a model to create embeddings, let's go ahead and embed some documents.

As we did in the previous example, we'll use Langchain's builit in loaders to read the documents from a directory.

In [ ]:
documents = DirectoryLoader(path=data_dir, glob="*.md", show_progress=True, loader_cls=UnstructuredMarkdownLoader).load()

The next step is to use a splitter. A splitter enables us to break up larger documents into chunks, so that we don't risk hitting the token limit when submitting our data to the embedding model.

In [ ]:
from langchain.text_splitter import CharacterTextSplitter
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
document_chunks = text_splitter.split_documents(documents)

The next stage is to convert the chunks of split documents into vectors which we do by passing the data through an embedding model. The resultant vectors are then stored in a vector database. In this example, we're using the Qdrant (pronounced 'quadrant') database (It's running in a container locally). We initialise it using the `location=":memory:"` option, so that the database will be stored in memory rather than persisted to disk.

In [ ]:
from langchain.vectorstores import Qdrant

qdrant = Qdrant.from_documents(
    document_chunks,
    embeddings_model,
    location=":memory:",
    collection_name="movies",
)

The above code segment handles the process of initialising the Qdrant database, passing our documents through the embedding model and storing the resulting vectors in the database.

Next, we define a retriever. In Langchain, retrievers are an interface that allow results to be returned from vector stores. So, we establish a retriever for our Qdrant database.

In [ ]:

retriever = qdrant.as_retriever()

Next we define a `RetrievalQA` chain. This chain brings handles the process of answering a question by performing the search on the vector store, then taking the results of that search and passing them to our AI model.

In [ ]:
from langchain.chains import RetrievalQA
qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=retriever)

Now, we'll run our query again. However, we'll make one small change.

You may be thinking that it's not surprising that the AI now knows about the latest Ant-Man movie, because we told it about the latest Ant-Man movie! So, let's try and show that the AI is actually doing some work here, after all it is a reasoning engine.

If you're not a fan of these movies, Ant-Man originates from Marvel comic books. And the collection of movies that originate from Marvel comic books are said to be part of the Marvel Cinematic Universe, sometimes referred to as the MCU. We haven't mentioned Marvel or MCU in the data we've provided, so if we modify the query slightly and ask the AI about the MCU instead of specifically about Ant-Man, it should be able to use reasoning to figure out what we mean.

In [ ]:
query = "Tell me about the latest MCU movie. When was it released? What is it about?"
qa.run(query)

If all went well, the AI should have responded that the latest MCU movie is Ant-Man and the Wasp: Quantumania which was released in February 2023.

So, we're getting the response we expected, but let's check in on one of the reasons why we've done all of this. Has the number of tokens used been reduced? Let's use the same technique as before and employ a callback to find out.

In [ ]:
with get_openai_callback() as callback:
    qa.run(query)
    total_tokens = callback.total_tokens

print(f"Total tokens used: {total_tokens}")

The exact number of tokens used may vary, but it should be clear that this query now uses far fewer tokens than our original query, typically around 2,000 fewer.

AI Orchestrators like Langchain and Semantic Kernel can help simplify the process of embedding, vectorization and search. In the preceding section, we stepped through the process of document splitting, embedding, vectorisation, storing vectors in a database and creating a retriever. In the next section, we use Langchain's document loader as we did previously to load and process our Markdown formatted documents, but this time we use a `VectorstoreIndexCreator` which you can see only requires a couple of parameters - the embedding model that we want to use and the source data (`loader`) to use. However, behind the scenes, the `VectorstoreIndexCreator` is carrying out all of the steps we did previously.

In [ ]:
from langchain.indexes import VectorstoreIndexCreator

loader = DirectoryLoader(path=data_dir, glob="*.md", show_progress=True, loader_cls=UnstructuredMarkdownLoader)

index = VectorstoreIndexCreator(
    embedding=embeddings_model
    ).from_loaders([loader])

Now, to run a query against our data, we just need to specify the prompt and then call the index we've created above and pass in the model (`llm`) we want to use and the question we want to ask.

In [ ]:
query = "Tell me about the latest Ant Man movie. When was it released? What is it about?"
index.query(llm=llm, question=query)

You can see this is a really simple way to implement embeddings and vectors as part of an AI application. It's great for getting up and running quickly.

We can use the callback method again to confirm that we're still seeing a reduced number of tokens being consumed.

In [ ]:
# Run the chain, using the callback to capture the number of tokens used
with get_openai_callback() as callback:
    index.query(llm=llm, question=query)
    total_tokens = callback.total_tokens

print(f"Total tokens used: {total_tokens}")